In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
train_data = pd.read_csv('datasets/titanic/train.csv')
test_data = pd.read_csv('datasets/titanic/test.csv')

In [4]:
test_data['FamilySize'] = 0
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch'] + 1

train_data['FamilySize'] = 0
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch'] + 1

In [5]:
combine = [train_data, test_data]

for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',
    'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1


In [7]:
# KnowCabin : 1 if known, 0 else
train_data['KnownCabin'] = 0
train_data.loc[train_data['Cabin'].isnull() == False, "KnownCabin"] = 1

test_data['KnownCabin'] = 0
test_data.loc[test_data['Cabin'].isnull() == False, "KnownCabin"] = 1

# Arbitrary change for the only person with missing value in Fare (7.5 since he is 3rd class)
test_data.loc[(test_data['Fare'].isnull() == True), 'Fare'] = 7.5

# Arbitrary change for the two persons without embarked data 
# ("C" because they both survived and got somewhat a frenchy name)
train_data.loc[train_data["Embarked"].isnull() == True, "Embarked"] = "C"

In [17]:
full_data = train_data.append(test_data)

full_data_knownage = full_data[full_data['Age'].notnull()]
full_data_age_unknown = full_data[full_data['Age'].isnull()]

/home/hernandezurbina/Dev/dltf/tf/lib/python3.5/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [18]:
full_data_knownage.head()

,Age,Cabin,Embarked,FamilySize,Fare,KnownCabin,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,NaN,S,2,7.2500,0,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,1
1,38.0,C85,C,2,71.2833,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,3
2,26.0,NaN,S,1,7.9250,0,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,2
3,35.0,C123,S,2,53.1000,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,3
4,35.0,NaN,S,1,8.0500,0,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,1


In [19]:
full_data_age_unknown.head()

,Age,Cabin,Embarked,FamilySize,Fare,KnownCabin,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title
5,NaN,NaN,Q,1,8.4583,0,"Moran, Mr. James",0,6,3,male,0,0.0,330877,1
17,NaN,NaN,S,1,13.0000,0,"Williams, Mr. Charles Eugene",0,18,2,male,0,1.0,244373,1
19,NaN,NaN,C,1,7.2250,0,"Masselmani, Mrs. Fatima",0,20,3,female,0,1.0,2649,3
26,NaN,NaN,C,1,7.2250,0,"Emir, Mr. Farred Chehab",0,27,3,male,0,0.0,2631,1
28,NaN,NaN,Q,1,7.8792,0,"O'Dwyer, Miss. Ellen ""Nellie""",0,29,3,female,0,1.0,330959,2
